In [1]:
import ee
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/vAFxyiIlnGuWGw6s6dq4H-NZYodDmRl7mbNdXjP3Sw_EN9YgEr4XlHI

Successfully saved authorization token.


In [2]:
import numpy as np
import xarray as xr
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import cartopy.crs as ccrs
import cartopy
import shapefile

ModuleNotFoundError: ignored

In [3]:
!apt-get -qq install python-cartopy python3-cartopy
import cartopy

Selecting previously unselected package python-pkg-resources.
(Reading database ... 135004 files and directories currently installed.)
Preparing to unpack .../0-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-pyshp.
Preparing to unpack .../1-python-pyshp_1.2.12+ds-1_all.deb ...
Unpacking python-pyshp (1.2.12+ds-1) ...
Selecting previously unselected package python-shapely.
Preparing to unpack .../2-python-shapely_1.6.4-1_amd64.deb ...
Unpacking python-shapely (1.6.4-1) ...
Selecting previously unselected package python-six.
Preparing to unpack .../3-python-six_1.11.0-2_all.deb ...
Unpacking python-six (1.11.0-2) ...
Selecting previously unselected package python-cartopy:amd64.
Preparing to unpack .../4-python-cartopy_0.14.2+dfsg1-2build3_amd64.deb ...
Unpacking python-cartopy:amd64 (0.14.2+dfsg1-2build3) ...
Selecting previously unselected package python3-pkg-resources.
Preparing to unpack .../5-pyth

In [0]:
#Import shapefiles


In [0]:
d = pd.read_csv('scripps_iceshelves.csv')
geo = d['.geo']
poly_list = []
#convert the list of string coordinates into a list of float -> ee.Polygon
for poly in np.arange(0,len(geo)):
    test_poly = geo[poly][35:-4].rsplit('],[');
    coord_list = []
    for substring in test_poly:
        [lon,lat] = substring.split(',')
        coord_list.append([float(lon.strip('[]')),float(lat.strip('[]'))])
    polygon = ee.Geometry.Polygon(coord_list)
    poly_list.append(polygon)
    

In [0]:
scripps_iceshelves = ee.FeatureCollection(poly_list)
#Individual ice shelves
amery = scripps_iceshelves.select([70]).geometry()

#Change here to pick different ice shelves
geometry_iceshelves = amery;

In [0]:
#Import Summer Landsat images (1-31 DOY, between 2015-1-1 to 2019-2-1, cloud cover <30%)
LandsatSummers = ee.ImageCollection('LANDSAT/LC08/C01/T2_TOA').select(['B2','B3','B4','B6','B8','B10']).filterDate('2015-01-01','2019-02-01').filterBounds(geometry_iceshelves).filter(ee.Filter.calendarRange(1,31,'day_of_year')).filterMetadata('CLOUD_COVER','less_than',0.3)

In [0]:
def makeLakeMask(image):
  #lake thresholds
    NDWI = image.normalizedDifference(['B2','B4']).gte(0.19)
    B2 = image.select('B2');
    B3 = image.select('B3');
    B4 = image.select('B4');
    B3B4difference = B3.subtract(B4).gte(0.07)
    B2B3difference = B2.subtract(B3).gte(0.07)
    lakethreshold = NDWI.bitwise_and(B3B4difference).bitwise_and(B2B3difference)

  #cloud mask
    B6 = image.select('B6')
    NDSI = image.normalizedDifference(['B3','B6'])
    NDSImask = NDSI.lt(0.8)
    B6mask = B6.gte(0.1)
    B2mask = B2.gte(0.6).bitwise_and(B2.lte(0.95))
    cloudmask = NDSImask.bitwise_and(B6mask).bitwise_and(B2mask)
  
  #rock mask
    B10 = image.select('B10');
    TIRSBlue = B10.divide(B2)
    TIRSBluemask = TIRSBlue.gte(0.65)
    B2mask = B2.lt(0.35)
    B4mask = B4.gt(0)
    rockmask = TIRSBluemask.bitwise_and(B2mask).bitwise_and(B4mask)

    LakeMask = lakethreshold.subtract(rockmask).subtract(cloudmask)
    return LakeMask


In [0]:
LakeMasks_LandsatSummers = LandsatSummers.map(makeLakeMask);
maxlakethresholdmap = LakeMasks_LandsatSummers.reduce('max')

Moussavi_lakes = maxlakethresholdmap.gt(0).clip(geometry_iceshelves)

In [0]:
#REMA import

REMAmosaic = ee.Image('UMN/PGC/REMA/V1_1/8m').select('elevation').divide(.001).clip(scripps_iceshelves).toInt64();

In [0]:
# Pick out Sinks
sinks = ee.Terrain.fillMinima(REMAmosaic)

           
filledSinks = sinks.subtract(REMAmosaic);

In [23]:
import folium 

def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Fetch an elevation model.
dem = ee.Image('USGS/SRTMGL1_003')
# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 100,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Create a folium map object.
my_map = folium.Map(location=[20, 0], zoom_start=3, height=500)

# Add the elevation model to the map object.
my_map.add_ee_layer(filledSinks, vis_params, 'Filled Sinks')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

EEException: ignored

In [24]:
filledSinks.
